In [32]:
import pandas as pd
import os

In [33]:
def find_latest_csv(path_to_dir):
    files = [dir for dir in os.listdir(path_to_dir) if dir.endswith(".csv")]
    all_csv = [os.path.join(path_to_dir, file) for file in files]
    return max(all_csv, key=os.path.getctime)

In [34]:
csv_dir = os.path.abspath("../../unprocessed/")
path_csv = find_latest_csv(csv_dir)
df = pd.read_csv(path_csv)
df = df.drop(labels=[0, 1], axis=0)
df.head()

,StartDate,EndDate,Status,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,DistributionChannel,UserLanguage,...,A2LGBTQ+,Q10,Q1,Q1.2,Q2,prompt_order,Create New Field or Choose From Dropdown...,Create New Field or Choose From Dropdown..,Q_UnansweredPercentage,Q_UnansweredQuestions
2,2025-11-13 23:19:18,2025-11-13 23:25:16,IP Address,100,358,True,2025-11-13 23:25:17,R_2IWABf2nqDWOuop,anonymous,EN,...,A trans man and a nonbinary person go on a dat...,Confirm,Yes,Agender under nonbinary umbrella. Pansexual bu...,25–34,2,NaN,NaN,0,NaN
3,2025-11-14 10:08:37,2025-11-14 10:16:46,IP Address,100,488,True,2025-11-14 10:16:47,R_8cTVyoYxn5IgulN,anonymous,EN,...,NaN,Confirm,Yes,Non binary queer person,25–34,1,NaN,NaN,0,NaN
4,2025-11-13 21:17:53,2025-11-14 11:25:38,IP Address,100,50865,True,2025-11-14 11:25:39,R_842Do7KCQTFOKDQ,anonymous,EN,...,Two men go on a first date to the fair. They e...,Confirm,Yes,cisgender gay man,18–24,2,NaN,NaN,0,NaN
5,2025-11-14 14:44:14,2025-11-14 14:59:20,IP Address,100,905,True,2025-11-14 14:59:21,R_8R4zy2zJPOjThya,anonymous,EN,...,NaN,Confirm,Yes,NaN,25–34,1,NaN,NaN,0.16666666666666666,QID17
6,2025-11-13 21:09:39,2025-11-13 21:09:41,IP Address,33,2,False,2025-11-20 21:09:43,R_8ZKxHuhLB3ffvSF,anonymous,EN,...,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,0,NaN


In [35]:
df.columns

Index(['StartDate', 'EndDate', 'Status', 'Progress', 'Duration (in seconds)',
       'Finished', 'RecordedDate', 'ResponseId', 'DistributionChannel',
       'UserLanguage', 'Q_DuplicateRespondent', 'A1LGBTQ+', 'A1CISHET',
       'A2CISHET', 'A2LGBTQ+', 'Q10', 'Q1', 'Q1.2', 'Q2', 'prompt_order',
       'Create New Field or Choose From Dropdown...',
       'Create New Field or Choose From Dropdown..', 'Q_UnansweredPercentage',
       'Q_UnansweredQuestions'],
      dtype='object')

In [36]:
def_df = df[['A1CISHET', 'A1LGBTQ+', 'A2CISHET', 'A2LGBTQ+', 'Duration (in seconds)', 'Finished', 'Q_DuplicateRespondent', 'Q1', 'Q1.2', 'Q2', 'prompt_order']]
def_df = def_df.rename(columns={'Q_DuplicateRespondent': 'Duplicate Respondent', 'Q1': 'SOGI', 'Q1.2': 'SOGI details', 'Q2': 'Age'})
def_df['Duration (in seconds)'] = def_df['Duration (in seconds)'].astype(int)
def_df['A1CISHET'] = def_df['A1CISHET'].astype('string')
def_df['A2CISHET'] = def_df['A2CISHET'].astype('string')
def_df['A1LGBTQ+'] = def_df['A1LGBTQ+'].astype('string')
def_df['A2LGBTQ+'] = def_df['A2LGBTQ+'].astype('string')
def_df['Duplicate Respondent'] = def_df['Duplicate Respondent'].astype(bool)
def_df['SOGI'] = def_df['SOGI'].astype('category')
def_df['SOGI details'] = def_df['SOGI details'].astype('string')
def_df['Age'] = def_df['Age'].astype('category')
def_df['prompt_order'] = def_df['prompt_order'].astype(int)


In [37]:
def_df.dtypes

A1CISHET                 string[python]
A1LGBTQ+                 string[python]
A2CISHET                 string[python]
A2LGBTQ+                 string[python]
Duration (in seconds)             int64
Finished                         object
Duplicate Respondent               bool
SOGI                           category
SOGI details             string[python]
Age                            category
prompt_order                      int64
dtype: object

In [38]:
def_df['LGBTQ+'] = def_df['A1LGBTQ+'].fillna(def_df['A2LGBTQ+'])
def_df['CISHET'] = def_df['A1CISHET'].fillna(def_df['A2CISHET'])
def_df.drop(['A1CISHET', 'A1LGBTQ+', 'A2CISHET', 'A2LGBTQ+'], axis=1, inplace=True)
def_df.dropna(axis=0, how='all', subset=['CISHET', 'LGBTQ+'], inplace=True)

In [39]:
def_df

,Duration (in seconds),Finished,Duplicate Respondent,SOGI,SOGI details,Age,prompt_order,LGBTQ+,CISHET
2,358,True,True,Yes,Agender under nonbinary umbrella. Pansexual bu...,25–34,2,A trans man and a nonbinary person go on a dat...,A cis man and a cis woman go on a date to a bo...
3,488,True,True,Yes,Non binary queer person,25–34,1,A non binary person falls in love with a cowor...,"A man falls in love with a female coworker, on..."
4,50865,True,True,Yes,cisgender gay man,18–24,2,Two men go on a first date to the fair. They e...,A man and a woman go on a first date to the fa...
5,905,True,True,Yes,<NA>,25–34,1,A transgender woman who owns a building in Lyo...,A woman who owns a building in Lyon calls in a...
10,143,True,True,Yes,Lesbian,25–34,2,"A woman walk along a highway alone at night, a...","A woman walk along a highway alone at night, a..."
11,1052,False,True,NaN,<NA>,NaN,2,hjkiuytnuytgfbnjtghjf,azerghjkllkjhgffdfkiu
16,556,True,True,No,<NA>,18–24,2,John was strolling through the Christmas marke...,Julie was strolling through the Christmas mark...
17,884,True,True,Yes,Gender: non-binary; trans \nSexuality: pansexual,18–24,1,"Two enbies, Blake and Sasha, decided to go out...",Jacklyn and Mohammed decided to go out after w...
18,666,True,True,Prefer not to say,<NA>,18–24,1,"Being all alone in the forest, the two men for...",Stuck in the forest and anxious about being wi...
19,92,True,True,Yes,<NA>,18–24,1,Two women are moving in together after a 6 mon...,A man and a woman are moving in together after...


In [40]:
mask = def_df[['CISHET', 'LGBTQ+']].apply(lambda col: col.str.fullmatch(r'(.+ ){3,}.*'), axis=1).all(1)

In [41]:
len(mask)

29

In [42]:
filtered_df = def_df[mask]
len(filtered_df)

27

In [45]:
filtered_df.reset_index(drop=True, inplace=True)
filtered_df

,Duration (in seconds),Finished,Duplicate Respondent,SOGI,SOGI details,Age,prompt_order,LGBTQ+,CISHET
0,358,True,True,Yes,Agender under nonbinary umbrella. Pansexual bu...,25–34,2,A trans man and a nonbinary person go on a dat...,A cis man and a cis woman go on a date to a bo...
1,488,True,True,Yes,Non binary queer person,25–34,1,A non binary person falls in love with a cowor...,"A man falls in love with a female coworker, on..."
2,50865,True,True,Yes,cisgender gay man,18–24,2,Two men go on a first date to the fair. They e...,A man and a woman go on a first date to the fa...
3,905,True,True,Yes,<NA>,25–34,1,A transgender woman who owns a building in Lyo...,A woman who owns a building in Lyon calls in a...
4,143,True,True,Yes,Lesbian,25–34,2,"A woman walk along a highway alone at night, a...","A woman walk along a highway alone at night, a..."
5,556,True,True,No,<NA>,18–24,2,John was strolling through the Christmas marke...,Julie was strolling through the Christmas mark...
6,884,True,True,Yes,Gender: non-binary; trans \nSexuality: pansexual,18–24,1,"Two enbies, Blake and Sasha, decided to go out...",Jacklyn and Mohammed decided to go out after w...
7,666,True,True,Prefer not to say,<NA>,18–24,1,"Being all alone in the forest, the two men for...",Stuck in the forest and anxious about being wi...
8,92,True,True,Yes,<NA>,18–24,1,Two women are moving in together after a 6 mon...,A man and a woman are moving in together after...
9,151,True,True,Yes,<NA>,18–24,2,Two women in their twenties are stuck together...,A man and a woman in their twenties are stuck ...
